In [ ]:
import os
from dotenv import load_dotenv
from langchain_community.cache import SQLiteCache
from langchain_core.globals import set_llm_cache
from langchain_teddynote import logging

# project 명
PROJECT = 'ocr-evaluation'

# project root 경로
PROJECT_ROOT_PATH = '.'

# default LLM 설정
DEFAULT_LLM = 'OPENAI' # 'UPSTAGE', 'OLLAMA'

# default embedding model 설정
DEFAULT_EMBEDDING_MODEL = 'OPENAI' # 'UPSTAGE', 'OLLAMA'

# coupon image 파일 경로
IMAGE_PATH = f'{PROJECT_ROOT_PATH}/data/coupon_image_files'

# OCR 모델에서 추출한 coupon 정보의 Ground Truth 정보가 저장된 파일 경로
COUPON_INFO_GT_JSON_PATH = f'{PROJECT_ROOT_PATH}/data/coupon_info_gt.json'

# Pororo OCR 모델에서 추출한 coupon 정보가 저장된 coupon_infos.json 파일 경로
COUPON_INFO_PORORO_JSON_PATH = f'{PROJECT_ROOT_PATH}/data/coupon_info_pororo.json'

# Upstage OCR 모델에서 추출한 coupon 정보가 저장된 coupon_infos_upstage.json 파일 경로
COUPON_INFO_UPSTAGE_JSON_PATH = f'{PROJECT_ROOT_PATH}/data/coupon_info_upstage.json'

# EasyOCR 모델에서 추출한 coupon 정보가 저장된 coupon_infos_easy.json 파일 경로
COUPON_INFO_EASY_JSON_PATH = f'{PROJECT_ROOT_PATH}/data/coupon_infos_easy.json'

# gpt-4o-mini 모델에서 추출한 coupon 정보가 저장된 coupon_infos.json 파일 경로
COUPON_INFO_GPT_4O_JSON_PATH = f'{PROJECT_ROOT_PATH}/data/coupon_info_gpt_4o.json'

# VectorDB 저장 경로
VECTOR_DB_ROOT_PATH = f'{PROJECT_ROOT_PATH}/.vector_db'

# FAISS 저장 경로
FAISS_DB_PATH = f'{VECTOR_DB_ROOT_PATH}/faiss'

# FAISS index 이름
FAISS_INDEX_NAME = 'coupon'

# Chroma 저장 경로
CHROMA_DB_PATH = f'{VECTOR_DB_ROOT_PATH}/chroma'

# Chroma collection 이름
CHROMA_COLLECTION_NAME = "coupon"

# cache 경로 설정
CACHE_PATH = f'{PROJECT_ROOT_PATH}/.cache'

# LLM CACHE 경로 설정
LLM_CACHE_PATH = f'{CACHE_PATH}/multimodal-rag_cache.db'

# env 설정 로딩
load_dotenv()

# 캐시 디렉토리를 생성합니다.
if not os.path.exists(CACHE_PATH):
    os.makedirs(CACHE_PATH)

# SQLiteCache를 사용합니다.
set_llm_cache(SQLiteCache(database_path=LLM_CACHE_PATH))

# langsmith 추적 시작
logging.langsmith(PROJECT)

## 1. OCR 모델별 coupon image로부터 추출 결과 로딩
- ground truth
- gpt-4o
- Pororo OCR
- Upstage OCR
- easyOCR

In [ ]:
import json 

# coupon_info_gt.json 로딩
with open(COUPON_INFO_GT_JSON_PATH, "r") as f:
	coupon_info_gts = json.load(f)
print(coupon_info_gts)

# Pororo OCR : coupon_info.json 로딩
with open(COUPON_INFO_PORORO_JSON_PATH, "r") as f:
	pororo_sources = json.load(f)
print(pororo_sources)

# Updstage OCR : coupon_info_upstage.json 로딩
with open(COUPON_INFO_UPSTAGE_JSON_PATH, "r") as f:
	upstage_sources = json.load(f)
print(upstage_sources)

# easy : coupon_info_easy.json 로딩
with open(COUPON_INFO_EASY_JSON_PATH, "r") as f:
	easy_sources = json.load(f)
print(easy_sources)

# gpt-4o : coupon_info_gpt_4o.json 로딩
with open(COUPON_INFO_GPT_4O_JSON_PATH, "r") as f:
	gpt_4o_sources = json.load(f)
print(gpt_4o_sources)

## 2. cosine similarity를 사용하여 ground truth와 유사도를 비교하기 위한 함수 선언
- get_ocr_model_result()

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from langchain_openai import OpenAIEmbeddings
import re

embedding = OpenAIEmbeddings(model="text-embedding-3-small")

def senitize(str):
    # print(str)
    tmp = re.sub(r"[^\uAC00-\uD7A30-9a-zA-Z\s]", " ", str)
    tmp = re.sub('상품명', "", tmp)
    tmp = re.sub(' +', " ", tmp)
    result = tmp.strip()
    # print(result)
    return result

# str = "['스타벅스 카페 아메리카노 T 2잔+클라우드 치즈 케이크','7616 0328 4683','스타벅스','교환처','유효기간','2022년 04월 28일','주문번호','1538761593','kakaotalk 선물하기']"
# str = "['상품명: 카페 아메리카노 T 2잔+클라우드 치즈 케이크, 쿠폰번호: 7616 0328 4683, 유효기간: 2022년 04월 28일, 교환처: 스타벅스, 주문번호: 1538761593']"
# senitize(str)

def similarity(a, b):
    return cosine_similarity([a], [b])[0][0]

def get_ocr_model_result(model, coupon_info_sources, coupon_info_gts):
    # ocr 결과 리스트
    ocr_sources = []

    # oct ground truth 리스트
    oct_gts = []

    for coupon_info_source in coupon_info_sources:
        # ocr_sources.append(coupon_info_source['coupon_info'])
        ocr_sources.append(senitize(coupon_info_source['coupon_info']))

        for coupon_info_gt in coupon_info_gts:
            if coupon_info_source['image_path'] == coupon_info_gt['image_path']:
                # oct_gts.append(coupon_info_gt['coupon_info'])
                oct_gts.append(senitize(coupon_info_gt['coupon_info']))
                break
    # print(ocr_sources)
    # print(oct_gts)

    embedded_sources = embedding.embed_documents(ocr_sources)
    embedded_gts = embedding.embed_documents(oct_gts)

    result_sum = 0

    for i, embedded_source in enumerate(embedded_sources):
        result = round(similarity(embedded_sources[i], embedded_gts[i]), 4)
        result_sum += result
        # print(f"[유사도 {result:.4f}] {ocr_sources[i]} \t <=====> \t {oct_gts[i]}")
        
    # 전체 유사도 합의 평균   
    result =  round((result_sum/len(embedded_sources)), 4)
    print(f"[model : {model}, 유사도 : {result:.4f}]")

    return {'model': model, 'similarity': result}



## 3. OCR 모델별 유사도 측정 실행
- 결과는 coupon 별 추출된 text를 ground truth와 비교한 유사도의 합을 coupon 수로 나눠 평균값을 구함

In [ ]:
model_similarity_results = []

# gpt-4o 모델 유사도 측정
gpt_4o_result = get_ocr_model_result('gpt-4o', gpt_4o_sources, coupon_info_gts)
model_similarity_results.append(gpt_4o_result)

# Pororo 모델 유사도 측정
pororo_result = get_ocr_model_result('Pororo OCR', pororo_sources, coupon_info_gts)
model_similarity_results.append(pororo_result)

# Upstage 모델 유사도 측정
upstage_result = get_ocr_model_result('Upstage OCR', upstage_sources, coupon_info_gts)
model_similarity_results.append(upstage_result)

# EasyOCR 모델 유사도 측정
easy_result = get_ocr_model_result('EasyOCR', easy_sources, coupon_info_gts)
model_similarity_results.append(easy_result)

print(model_similarity_results)

## 4. 실험 결과 : Pororo OCR 선정

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

x_value = []
y_value = []
for model_similarity_result in model_similarity_results:
    x_value.append(model_similarity_result['model'])
    y_value.append(model_similarity_result['similarity'])

x = np.arange(4)

# plt.bar(x, values, color='y')

plt.xlabel('OCR Model')
plt.ylabel('Similarity')

plt.bar(x, y_value, color='dodgerblue', width=0.4)
# plt.bar(x, values, color='C2')
# plt.bar(x, y_value, color='#e35f62', width=0.4)
plt.xticks(x, x_value)
plt.text(0, y_value[0], y_value[0])
plt.text(1, y_value[1], y_value[1])
plt.text(2, y_value[2], y_value[2])
plt.text(3, y_value[3], y_value[3])

plt.show()

한국어, 영어 같은 문장 similarity 테스트

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from langchain_openai import OpenAIEmbeddings

embedding = OpenAIEmbeddings(model="text-embedding-3-small")

def similarity(a, b):
    return cosine_similarity([a], [b])[0][0]

# 0.571
# source = "What are the product codes and expiration dates for the items available at GS25?"

# 0.6737
source = "GS25에서 판매되는 상품의 상품 코드와 유통기한은 어떻게 되나요?"
# gt = "GS25에서 사용할 수 있는 5천원 모바일 상품권 쿠폰의 코드 번호와 유효기간은 무엇인가요?"

# 0.644
gt = "What are the product codes and expiration dates for the items available at GS25?"

embedded_source = embedding.embed_query(source)
embedded_gt = embedding.embed_query(gt)

result = round(similarity(embedded_source, embedded_gt), 4)
print(result)